In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import ttest_ind, f_oneway
import scipy.stats as stats

import sys
from pathlib import Path

# # Add src directory to the Python path
sys.path.append(str(Path.cwd().parent))

from src.data_analysis.df_dataattribute_analysis import categorize_columns
from src.data_sourcing.import_export_data import get_data, save_data

from collections import defaultdict
from typing import List, Dict, Tuple

df = get_data('featured_engineered_testing.csv', 'processed_data')
df.head(4)


,ATL_OR_DR,CAMPAIGN_TYPE,CHANNEL,COST,FREE_TRIALS,HOLIDAY_FLAG,SEASON,MONTH,ATL_OR_DR_CAMPAIGN_TYPE,ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL,...,LOG_COST,LOG_FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,AVG_COST_MONTH,MEDIAN_COST_MONTH,AVG_COST_CAMPAIGN_TYPE_CHANNEL,MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL,AVG_FREE_TRIALS_MONTH,MEDIAN_FREE_TRIALS_MONTH
0,DR - Direct Response,Title,paid social,7784.31,86401.15,False,Summer,June,DR - Direct Response_Title,DR - Direct Response_Title_paid social,...,8.959994,11.366768,0.086265,20.326622,16887.563506,3643.265,14781.334567,3309.50,42865.334723,25054.485
1,DR - Direct Response,Title,app,2474.31,2956.74,False,Summer,August,DR - Direct Response_Title,DR - Direct Response_Title_app,...,7.814121,7.992181,0.608045,15.805560,14172.931604,2955.060,15454.451540,3477.08,43953.768382,28127.390
2,ATL - Above The Line,Title,paid social,10222.82,12513.08,False,Summer,July,ATL - Above The Line_Title,ATL - Above The Line_Title_paid social,...,9.232476,9.434610,0.597171,18.666908,13667.902515,2994.470,14781.334567,3309.50,45959.687637,27091.210
3,DR - Direct Response,Title,app,49631.87,17207.04,False,Winter,December,DR - Direct Response_Title,DR - Direct Response_Title_app,...,10.812409,9.753132,1.356967,20.565462,18937.983302,3240.025,15454.451540,3477.08,47495.480279,28640.060


### Feature Aggregation with Channel and Month with Free Trials using Mean and Median

In [2]:
# Average/Median FREE_TRIALS generated per CHANNEL in each MONTH.
categories = ['CHANNEL', 'ATL_OR_DR', 'CAMPAIGN_TYPE']
numericals = ['FREE_TRIALS', 'COST_PER_FREE_TRIALS', 'COST']
for category in categories:
    for numerical in numericals:
        #df.groupby(['CHANNEL', 'MONTH'])['FREE_TRIALS'].agg(['mean', 'median']).reset_index()
        column_name = f'AVG_{numerical}_BY_{category}_MONTH'
        df[column_name] = df.groupby([category, 'MONTH'])[numerical].transform('mean')
        df[f'MEDIAN_{numerical}_BY_{category}_MONTH'] = df.groupby([category, 'MONTH'])[numerical].transform('median')

        # Channel-specific monthly performance metrics
        df[f'{numerical}_OVER_{column_name}'] = df[numerical] /df[column_name]

# Perform mean normalization
df["NORMALIZE_FREE_TRIAL_BY_MONTH"] = df["FREE_TRIALS"] / df.groupby("MONTH")["FREE_TRIALS"].transform("mean")
df["NORMALIZE_COST_PER_FREE_TRIALS_BY_MONTH"] = df["COST_PER_FREE_TRIALS"] / df.groupby("MONTH")["COST_PER_FREE_TRIALS"].transform("mean")
df["NORMALIZE_COST_BY_FREE_TRIALS_BY_MONTH"] = df["COST_BY_FREE_TRIALS"] / df.groupby("MONTH")["COST_BY_FREE_TRIALS"].transform("mean")


In [3]:
save_data(df, 'featured_engineered_aggregations.csv', 'processed_data')

2025-01-30 23:00:55,282 - INFO - Data saved successfully!
Environment: development
Data Type: processed_data
File Path: C:\Users\cheng\Workspace\Paramount\data\processed\featured_engineered_aggregations.csv
Rows Saved: 14108


In [4]:
# def generate_stats_count(categorical_data, quantitative_data):
#     # Store all values for each category
#     values_by_category = defaultdict(list)

#     # Collect all values for each category
#     for cd, qd in zip(categorical_data, quantitative_data):
#         values_by_category[cd].append(qd)

#     # Calculate statistics
#     stat_hashmap = defaultdict(dict)
#     mean_output = []
#     median_output = []

#     # Calculate stats for each category
#     for key in values_by_category:
#         values = values_by_category[key]
#         stat_hashmap[key]['mean'] = np.mean(values)  # or sum(values)/len(values)
#         stat_hashmap[key]['median'] = np.median(values)

#     # Create output lists in same order as input
#     for cd in categorical_data:
#         mean_output.append(stat_hashmap[cd]['mean'])
#         median_output.append(stat_hashmap[cd]['median'])

#     return mean_output, median_output
# # Generate stats for the 'CHANNEL' columm
# mean_data, median_data = generate_stats_count(df['MONTH'], df['COST_BY_FREE_TRIALS'])
# df['AVG_COST_BY_FREE_TRIALS_BY_MONTH_2'] = mean_data
# df['MEDIAN_COST_BY_FREE_TRIALS_BY_MONTH_2'] = median_data

# # def generate_stats_count_error(categorical_data, quantitative_data):
# #     hashmap_categorical = defaultdict(int)
# #     hashmap_quantitative = defaultdict(float)
# #     for cd, qd in zip(categorical_data, quantitative_data):
# #         if cd not in hashmap_categorical:
# #             hashmap_categorical[cd] = 1
# #             hashmap_quantitative[cd] = qd
# #         else:
# #             hashmap_categorical[cd] += 1
# #             hashmap_quantitative[cd] += qd

# #     print(hashmap_quantitative)
# #     stat_hashmap = defaultdict(dict)
# #     for key in hashmap_categorical.keys():
# #         stat_hashmap[key]['mean'] = hashmap_quantitative[key] / hashmap_categorical[key]
# #         stat_hashmap[key]['median'] = np.median(hashmap_quantitative[key])

# #     mean_output, median_output = [], []
# #     for cd in categorical_data:
# #         mean_output.append(stat_hashmap[cd]['mean'])
# #         median_output.append(stat_hashmap[cd]['median'])
# #     return mean_output, median_output

In [5]:
# # Mean Normalization by Month:  FREE_TRIALS / Average FREE_TRIALS in that MONTH.
# # This helps adjust for seasonal patterns without assuming campaign continuity.
# df["MEAN_NORM_BY_MONTH_FREE_TRIALS"] = df["FREE_TRIALS"] / df.groupby("MONTH")["FREE_TRIALS"].transform("mean")

# # def calculate_mean_norm_by_month(months: List[str], free_trials: List[float]) -> List[float]:
# #     # Step 1: Calculate monthly sums and counts in a single pass
# #     # Using defaultdict to avoid key existence checks
# #     monthly_sums = defaultdict(float)
# #     monthly_counts = defaultdict(int)

# #     for month, trials in zip(months, free_trials):
# #         monthly_sums[month] += trials
# #         monthly_counts[month] += 1

# #     # Step 2: Calculate monthly averages
# #     # Store in dict for O(1) lookup during normalization
# #     monthly_averages = {
# #         month: monthly_sums[month] / monthly_counts[month]
# #         for month in monthly_sums
# #     }

# #     # Step 3: Calculate normalized values
# #     # Using list comprehension for better performance than append operations
# #     normalized_values = [
# #         trials / monthly_averages[month]
# #         for month, trials in zip(months, free_trials)
# #     ]

# #     return normalized_values


# # # Convert DataFrame columns to lists for processing
# # months = df['MONTH'].tolist()
# # free_trials = df['FREE_TRIALS'].tolist()

# # # Use the function from the previous code
# # normalized_values = calculate_mean_norm_by_month(months, free_trials)
# # df['test'] = normalized_values


### Month-over-Month percent change for FREE_TRIALS using Pandas Shift == LAG/LEAD in SQL

In [6]:
# 🟠 Month-over-Month (MoM) Percent Change for FREE_TRIALS
# 🟢 Always be aware of ordering. This ain't SQL!
# df = df.sort_values(['CHANNEL', 'REPORT_DATE'])

# df['PREV_MONTH_FREE_TRIALS'] = df.groupby('CHANNEL')['FREE_TRIALS'].shift(1)
# df['mom_trials_change'] = (df['FREE_TRIALS'] - df['prev_month_trials']) / df['prev_month_trials']

# # 🟠 Month-over-Month (MoM) Percent Change for COST
# df['prev_month_cost'] = df.groupby('CHANNEL')['COST'].shift(1)
# df['mom_cost_change'] = (df['COST'] - df['prev_month_cost']) / df['prev_month_cost']

In [7]:
# #df['Rolling_Sum'] = df.groupby('Category')['Value'].rolling(3, min_periods=1).sum().reset_index(level=0, drop=True)
# # 3-month rolling average for FREE_TRIALS
# df['Rolling_3m_FREE_TRIALS'] = df.groupby('CHANNEL')['FREE_TRIALS'].rolling(3, min_periods=1).mean().reset_index(level=0, drop=True)

# # 3-month rolling average for COST
# df['Rolling_3m_COST'] = df.groupby('CHANNEL')['COST'].rolling(3, min_periods=1).mean().reset_index(level=0, drop=True)


In [8]:
# # Compute MoM Growth by CAMPAIGN_TYPE
# df['prev_month_trials_campaign'] = df.groupby(['CAMPAIGN_TYPE', 'MONTH'])['FREE_TRIALS'].shift(1)
# df['Growth_FREE_TRIALS_Campaign'] = (df['FREE_TRIALS'] - df['prev_month_trials_campaign']) / df['prev_month_trials_campaign']

# # Compute MoM Growth by CHANNEL
# df['prev_month_trials_channel'] = df.groupby(['CHANNEL', 'MONTH'])['FREE_TRIALS'].shift(1)
# df['Growth_FREE_TRIALS_Channel'] = (df['FREE_TRIALS'] - df['prev_month_trials_channel']) / df['prev_month_trials_channel']


In [9]:
# # Growth Rate by ATL_OR_DR
# df['prev_month_trials_atl_dr'] = df.groupby(['ATL_OR_DR', 'MONTH'])['FREE_TRIALS'].shift(1)
# df['Growth_FREE_TRIALS_ATL_DR'] = (df['FREE_TRIALS'] - df['prev_month_trials_atl_dr']) / df['prev_month_trials_atl_dr']

# # Growth Rate by HOLIDAY_FLAG
# df['prev_month_trials_holiday'] = df.groupby(['HOLIDAY_FLAG', 'MONTH'])['FREE_TRIALS'].shift(1)
# df['Growth_FREE_TRIALS_Holiday'] = (df['FREE_TRIALS'] - df['prev_month_trials_holiday']) / df['prev_month_trials_holiday']


### Friendship Timeline Meta

In [10]:
# https://www.interviewquery.com/questions/friendship-timeline?utm_medium=email&utm_source=weekly_question
friends_added = [
    {'user_ids': [1, 2], 'created_at': '2020-01-01'},
    {'user_ids': [3, 2], 'created_at': '2020-01-02'},
    {'user_ids': [2, 1], 'created_at': '2020-02-02'},
    {'user_ids': [4, 1], 'created_at': '2020-02-02'}]

friends_removed = [
    {'user_ids': [2, 1], 'created_at': '2020-01-03'},
    {'user_ids': [2, 3], 'created_at': '2020-01-05'},
    {'user_ids': [1, 2], 'created_at': '2020-02-05'}]

In [11]:

from collections import defaultdict, deque
hashmap_start, hashmap_end = defaultdict(deque), defaultdict(deque)
friends_added = sorted(friends_added, key= lambda x: x['created_at'])

friends_removed = sorted(friends_removed, key= lambda x: x['created_at'])
friends_removed

[{'user_ids': [2, 1], 'created_at': '2020-01-03'},
 {'user_ids': [2, 3], 'created_at': '2020-01-05'},
 {'user_ids': [1, 2], 'created_at': '2020-02-05'}]

In [12]:
for i in range(min(len(friends_added), len(friends_removed))):
    added = friends_added[i]
    removed = friends_removed[i]

    add_user_a, add_user_b = added['user_ids']
    add_created_at = added['created_at']

    remove_user_a, remove_user_b = removed['user_ids']
    remove_created_at = removed['created_at']

    if add_user_a > add_user_b:
        add_user_a, add_user_b = add_user_b, add_user_a


    hashmap_start[(add_user_a,add_user_b)].append(add_created_at)

    if remove_user_a > remove_user_b:
        remove_user_a, remove_user_b = remove_user_b, remove_user_a

    hashmap_end[(remove_user_a, remove_user_b)].append(remove_created_at)

if len(friends_added) > len(friends_removed):
    for j in range(i, len(friends_added)):
        added = friends_added[j]
        add_user_a, add_user_b = added['user_ids']
        add_created_at = added['created_at']
        if add_user_a > add_user_b:
            add_user_a, add_user_b = add_user_b, add_user_a
        hashmap_start[(add_user_a, add_user_b)].append(add_created_at)




In [13]:
hashmap_end

defaultdict(collections.deque,
            {(1, 2): deque(['2020-01-03', '2020-02-05']),
             (2, 3): deque(['2020-01-05'])})

In [14]:
output = []

for key, queue in hashmap_end.items():
    while queue:
        end_date = queue.popleft()
        start_date = hashmap_start[key].popleft()

        output.append({
            'user_ids': list(key),
            'start_date': start_date,
            'end_date': end_date
        })




In [15]:
output

[{'user_ids': [1, 2], 'start_date': '2020-01-01', 'end_date': '2020-01-03'},
 {'user_ids': [1, 2], 'start_date': '2020-02-02', 'end_date': '2020-02-05'},
 {'user_ids': [2, 3], 'start_date': '2020-01-02', 'end_date': '2020-01-05'}]